In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np


2024-10-02 12:11:43.995292: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-02 12:11:44.002830: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-02 12:11:44.069917: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-02 12:11:44.121324: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 12:11:44.164394: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
#Collect data
import nltk 
nltk.download('gutenberg')
from nltk.corpus import gutenberg

# Load the Hamlet text
data = gutenberg.raw('shakespeare-hamlet.txt')

with open('hamlet.txt', 'w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     /home/sikander/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [3]:
#Data preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Load the data
with open('hamlet.txt', 'r') as file:
    text = file.read().lower()

#Tokeinze the text    
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1


In [4]:
#Create input sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
input_sequences

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [57, 407, 2, 1181, 177, 1892],
 [407, 1182],
 [407, 1182, 63],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21],
 [408, 162, 377, 21, 247],
 [408, 162, 377, 21, 247, 882],
 [18, 66],
 [451, 224],
 [451, 224, 248],
 [451, 224, 248, 1],
 [451, 224, 248, 1, 30],
 [408, 407],
 [451, 25],
 [408, 6],
 [408, 6, 43],
 [408, 6, 43, 62],
 [408, 6, 43, 62, 1893],
 [408, 6, 43, 62, 1893, 96],
 [408, 6, 43, 62, 1893, 96, 18],
 [408, 6, 43, 62, 1893, 96, 18, 566],
 [451, 71],
 [451, 71, 51],
 [451, 71, 51, 1894],
 [451, 71, 51, 1894, 567],
 [451, 71, 51, 1894, 567, 378],
 [451, 71, 51, 1894, 567, 378, 80],
 [451, 71, 51, 1894, 567, 378, 80, 3],
 [451, 71, 51, 1894, 567, 378, 80, 3, 273],
 [451, 71

In [14]:
#pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [15]:
#Create predictors and labels
import tensorflow as tf 
x, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)


In [8]:
#Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [41]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


inputs = Input(shape=(max_sequence_len-1,))
x = Embedding(total_words, 100)(inputs)
x = LSTM(150, return_sequences=True)(x)
x = Dropout(0.2)(x)
x = LSTM(100)(x)
outputs = Dense(total_words, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

total_words, max_sequence_len


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 13)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_19 (Embedding)        │ (None, 13, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_36 (LSTM)                  │ (None, 13, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 13, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_37 (LSTM)                  │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 4818)           │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,219,418 (4.65 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

(4818, 14)

In [43]:
#Train the model
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), verbose=2)

Epoch 1/50
644/644 - 12s - 19ms/step - accuracy: 0.0643 - loss: 5.8861 - val_accuracy: 0.0608 - val_loss: 6.8765
Epoch 2/50
644/644 - 12s - 19ms/step - accuracy: 0.0723 - loss: 5.7334 - val_accuracy: 0.0674 - val_loss: 6.9302
Epoch 3/50
644/644 - 12s - 19ms/step - accuracy: 0.0792 - loss: 5.5904 - val_accuracy: 0.0690 - val_loss: 7.0174
Epoch 4/50
644/644 - 12s - 19ms/step - accuracy: 0.0862 - loss: 5.4576 - val_accuracy: 0.0690 - val_loss: 7.0756
Epoch 5/50
644/644 - 14s - 21ms/step - accuracy: 0.0954 - loss: 5.3283 - val_accuracy: 0.0699 - val_loss: 7.1898
Epoch 6/50
644/644 - 13s - 21ms/step - accuracy: 0.1005 - loss: 5.2030 - val_accuracy: 0.0723 - val_loss: 7.2540
Epoch 7/50
644/644 - 13s - 21ms/step - accuracy: 0.1064 - loss: 5.0788 - val_accuracy: 0.0680 - val_loss: 7.3712
Epoch 8/50
644/644 - 13s - 21ms/step - accuracy: 0.1112 - loss: 4.9591 - val_accuracy: 0.0717 - val_loss: 7.4698
Epoch 9/50
644/644 - 13s - 21ms/step - accuracy: 0.1174 - loss: 4.8349 - val_accuracy: 0.0672 - 

In [47]:
#Function to predict the next word
def predict_next_word(input_text, model, tokenizer, max_sequence_len):
    input_sequence = tokenizer.texts_to_sequences([input_text])[0]
    input_sequence = pad_sequences([input_sequence], maxlen=max_sequence_len-1, padding='pre')
    prediction = model.predict(input_sequence, verbose=0)
    index = np.argmax(prediction[0])
    word = tokenizer.index_word[index]
    return word

In [54]:
input_text = 'to be or not to be considered as'
predict_next_word(input_text, model, tokenizer, max_sequence_len)


'i'